# Crypto Machine‑Learning Trading Strategy 🚀

This notebook adapts the Backtesting.py example *“Trading with Machine Learning”* to four cryptocurrencies (**ADA, BTC, ETH, SOL**) using OHLCV data from 2024‑2025.

> **Reference example:** <https://kernc.github.io/backtesting.py/doc/examples/Trading%20with%20Machine%20Learning.html>

---

Each CSV is assumed to include:

| column | description          |
|--------|----------------------|
| `date_only` | `YYYY‑MM‑DD`       |
| `time_only` | `HH:MM:SS`         |
| `open, high, low, close` | OHLC prices |
| `volume` | traded volume        |

Adjust the loader if your schema differs.


In [1]:
!pip install backtesting ta scikit-learn pandas numpy matplotlib
# suppress warnings
import warnings
warnings.filterwarnings('ignore')
%pip show ta

Name: ta
Version: 0.11.0
Summary: Technical Analysis Library in Python
Home-page: https://github.com/bukosabino/ta
Author: Dario Lopez Padial (Bukosabino)
Author-email: Bukosabino@gmail.com
License: The MIT License (MIT)
Location: /Users/jadenfix/.pyenv/versions/3.11.3/lib/python3.11/site-packages
Requires: numpy, pandas
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys

from ta.volatility import BollingerBands
from ta.trend import SMAIndicator
from ta.momentum import RSIIndicator, StochasticOscillator

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 1. Imports for extra models
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from hmmlearn.hmm import GaussianHMM

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Flatten

from backtesting import Backtest, Strategy
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)


Loading BokehJS ...

## 1  Load data

In [3]:
# 🔗 Update the paths for your local machine
CSV_PATHS = {
    'ADA': Path('/Users/jadenfix/hedge-fund-in-a-box/cpp_engine/data/2024_2025/2024_to_april_2025_ada_data.csv'),
    'BTC': Path('/Users/jadenfix/hedge-fund-in-a-box/cpp_engine/data/2024_2025/2024_to_april_2025_btc_data.csv'),
    'ETH': Path('/Users/jadenfix/hedge-fund-in-a-box/cpp_engine/data/2024_2025/2024_to_april_2025_eth_data.csv'),
    'SOL': Path('/Users/jadenfix/hedge-fund-in-a-box/cpp_engine/data/2024_2025/2024_to_april_2025_solana_data.csv'),
}

def load_crypto(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    df['datetime'] = pd.to_datetime(df['date_only'] + ' ' + df['time_only'])
    df = df.set_index('datetime').sort_index()
    df = df.rename(columns=str.lower)[['open', 'high', 'low', 'close', 'volume']]
    return df

crypto_dfs = {sym: load_crypto(p) for sym, p in CSV_PATHS.items()}
crypto_dfs['BTC'].head()


,open,high,low,close,volume
datetime,,,,,
2024-01-01 00:00:00,42351.14,42364.78,42314.01,42361.18,46.06648
2024-01-01 00:01:00,42361.36,42388.00,42357.51,42378.00,45.68457
2024-01-01 00:02:00,42377.20,42400.00,42373.47,42391.03,31.84134
2024-01-01 00:03:00,42391.02,42436.68,42388.98,42433.83,48.97387
2024-01-01 00:04:00,42435.90,42468.62,42433.54,42464.90,40.65900


## 2  Feature engineering

In [4]:
def add_indicators(df: pd.DataFrame,
                    window_short=14,
                    window_long=50) -> pd.DataFrame:
    df = df.copy()
    close = df['close']

    df['sma_short'] = SMAIndicator(close, window_short).sma_indicator()
    df['sma_long']  = SMAIndicator(close, window_long).sma_indicator()
    df['rsi']       = RSIIndicator(close, window=14).rsi()

    bb = BollingerBands(close, window=20, window_dev=2)
    df['bb_high']   = bb.bollinger_hband()
    df['bb_low']    = bb.bollinger_lband()

    stoch = StochasticOscillator(df['high'], df['low'], close, window=14)
    df['stoch']     = stoch.stoch()

    # Label: next-period direction
    df['return']    = close.pct_change().shift(-1)
    df['direction'] = np.sign(df['return']).replace(0, np.nan).fillna(method='bfill')
    df.dropna(inplace=True)
    return df

crypto_dfs = {s: add_indicators(d) for s, d in crypto_dfs.items()}
crypto_dfs['BTC'].head()


,open,high,low,close,volume,sma_short,sma_long,rsi,bb_high,bb_low,stoch,return,direction
datetime,,,,,,,,,,,,,
2024-01-01 00:49:00,42475.57,42477.16,42462.18,42466.00,10.23624,42485.025000,42485.2198,46.146817,42529.279057,42409.603943,47.124304,0.000403,1.0
2024-01-01 00:50:00,42466.37,42489.62,42461.81,42483.12,13.11180,42488.377857,42487.6586,50.616404,42530.348220,42412.406780,54.748667,0.000042,1.0
2024-01-01 00:51:00,42482.41,42492.36,42473.36,42484.89,14.11998,42489.290000,42489.7964,51.068570,42531.275418,42415.575582,40.116279,0.000030,1.0
2024-01-01 00:52:00,42482.93,42492.36,42482.93,42486.16,3.37736,42490.712143,42491.6990,51.412331,42530.464387,42422.219613,42.111879,0.000120,1.0
2024-01-01 00:53:00,42486.40,42492.36,42481.60,42491.27,2.58677,42489.520000,42492.8478,52.847732,42530.107794,42428.236206,50.141420,0.000391,1.0


## 3  Train/test split & model training (BTC example)

In [5]:
symbol = 'BTC'
df = crypto_dfs[symbol]

FEATURES = ['sma_short', 'sma_long', 'rsi', 'bb_high', 'bb_low', 'stoch']
X, y = df[FEATURES], df['direction']

split = int(len(df) * 0.8)
X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

rf = RandomForestClassifier(n_estimators=300, max_depth=6,
                            class_weight='balanced', random_state=42)
rf.fit(X_train, y_train)
print(classification_report(y_test, rf.predict(X_test)))


              precision    recall  f1-score   support

        -1.0       0.50      0.81      0.62     69804
         1.0       0.52      0.21      0.30     70154

    accuracy                           0.51    139958
   macro avg       0.51      0.51      0.46    139958
weighted avg       0.51      0.51      0.46    139958



## 4  Define Backtesting strategy

In [9]:
class MLStrategy(Strategy):
    model = rf
    feats = FEATURES

    def init(self):
        # Nothing special to initialize in this simple example,
        # but Backtesting.py requires this method to exist.
        pass

    def next(self):
        if len(self.data) < len(self.feats):
            return

        # grab the latest indicator values
        row = [self.data.df[f].iloc[-1] for f in self.feats]
        pred = self.model.predict([row])[0]

        if pred > 0 and not self.position.is_long:
            self.position.close()
            self.buy()
        elif pred < 0 and not self.position.is_short:
            self.position.close()
            self.sell()

## 5  Backtest

In [ ]:
# Say you’re backtesting BTC:
df_bt = crypto_dfs['BTC'].copy()

# 1. Rename columns to capitalized OHLCV
df_bt = df_bt.rename(columns={
    'open':   'Open',
    'high':   'High',
    'low':    'Low',
    'close':  'Close',
    'volume': 'Volume'
})

# 2. Drop any rows missing those OHLCV values
df_bt = df_bt.dropna(subset=['Open','High','Low','Close','Volume'])

# 3. (Optional) Quick sanity-check
print(df_bt.columns)
# Index(['Open','High','Low','Close','Volume',  ... your indicator columns ...], dtype='object')

# 4. Run the backtest
bt = Backtest(
    df_bt,
    MLStrategy,
    cash=10_000,
    commission=0.001,
    exclusive_orders=True
)
stats = bt.run()
print(stats)
bt.plot(figsize=(14,6))

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'sma_short', 'sma_long',
       'rsi', 'bb_high', 'bb_low', 'stoch', 'return', 'direction'],
      dtype='object')


In [ ]:
bt.plot(figsize=(14, 6))

## 6  Next steps

* Walk‑forward retraining
* Hyper‑parameter tuning
* Probability‑based position sizing
* Portfolio‑level allocation across multiple crypto pairs
* Stop‑loss / take‑profit management

---